In [1]:
import pygame
import pygame_widgets
from pygame_widgets.dropdown import Dropdown
import sys
import json

# Initialize Pygame
pygame.init()

# Constants
GRID_SIZE = 20  # 20x20 grid
CELL_SIZE = 30  # Each cell is 30x30 pixels
TURN_RADIUS = 3

WIDTH = GRID_SIZE * CELL_SIZE + 200
HEIGHT = GRID_SIZE * CELL_SIZE + 200  # Extra space for buttons

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
LIGHT_BLUE = (173, 216, 230)
RED = (255, 0, 0)
TEXT_COLOR = (0, 0, 0)

# Creating window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Robot Car Simulator')



# List to store coordinates of obstacles
obstacles = []

# Font for rendering text
font = pygame.font.Font(None, CELL_SIZE)

# Directions (0 = North, 1 = East, 2 = South, 3 = West)
current_direction = 0  # Initially facing North

# Starting position of the car (bottom-left corner)
car_x, car_y = 0, GRID_SIZE - 3

# Robot car size
robot_car_size = 3

# Loading robot car image
robot_car_image = pygame.image.load('robotcar.jpg')
robot_car_image = pygame.transform.scale(robot_car_image, (CELL_SIZE * 3, CELL_SIZE * 3))  # Resize to 3x3 cells

# Function to draw the grid
def draw_grid():
    for x in range(100, WIDTH - 100, CELL_SIZE):
        for y in range(50, HEIGHT - 150, CELL_SIZE):  # Leave space for buttons
            rect = pygame.Rect(x, y, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, BLACK, rect, 1)

# Function to draw the robot car image
def draw_car():
    # Rotate the car image based on the current direction
    rotated_image = pygame.transform.rotate(robot_car_image, -current_direction * 90)  # Rotate clockwise
    screen.blit(rotated_image, (car_x * CELL_SIZE + 100, car_y * CELL_SIZE + 50))

class Button(object):
    def __init__(self,rect,command,**kwargs):
        self.rect = pygame.Rect(rect)
        self.command = command
        self.clicked = False
        self.hovered = False
        self.hover_text = None
        self.clicked_text = None
        self.process_kwargs(kwargs)
        self.render_text()
 
    def process_kwargs(self,kwargs):
        settings = {
            "color"             : pygame.Color('red'),
            "text"              : None,
            "font"              : None, #pg.font.Font(None,16),
            "call_on_release"   : True,
            "hover_color"       : None,
            "clicked_color"     : None,
            "font_color"        : pygame.Color("white"),
            "hover_font_color"  : None,
            "clicked_font_color": None,
            "click_sound"       : None,
            "hover_sound"       : None,
            'border_color'      : pygame.Color('black'),
            'border_hover_color': pygame.Color('yellow'),
            'disabled'          : False,
            'disabled_color'     : pygame.Color('grey'),
            'radius'            : 3,
        }
        for kwarg in kwargs:
            if kwarg in settings:
                settings[kwarg] = kwargs[kwarg]
            else:
                raise AttributeError("{} has no keyword: {}".format(self.__class__.__name__, kwarg))
        self.__dict__.update(settings)
 
    def render_text(self):
        if self.text:
            if self.hover_font_color:
                color = self.hover_font_color
                self.hover_text = self.font.render(self.text,True,color)
            if self.clicked_font_color:
                color = self.clicked_font_color
                self.clicked_text = self.font.render(self.text,True,color)
            self.text = self.font.render(self.text,True,self.font_color)
 
    def get_event(self,event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            self.on_click(event)
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            self.on_release(event)
 
    def on_click(self,event):
        if self.rect.collidepoint(event.pos):
            self.clicked = True
            if not self.call_on_release:
                self.function()
 
    def on_release(self,event):
        if self.clicked and self.call_on_release:
            #if user is still within button rect upon mouse release
            if self.rect.collidepoint(pygame.mouse.get_pos()):
                self.command()
        self.clicked = False
 
    def check_hover(self):
        if self.rect.collidepoint(pygame.mouse.get_pos()):
            if not self.hovered:
                self.hovered = True
                if self.hover_sound:
                    self.hover_sound.play()
        else:
            self.hovered = False
 
    def draw(self,surface):
        color = self.color
        text = self.text
        border = self.border_color
        self.check_hover()
        if not self.disabled:
            if self.clicked and self.clicked_color:
                color = self.clicked_color
                if self.clicked_font_color:
                    text = self.clicked_text
            elif self.hovered and self.hover_color:
                color = self.hover_color
                if self.hover_font_color:
                    text = self.hover_text
            if self.hovered and not self.clicked:
                border = self.border_hover_color
        else:
            color = self.disabled_color
         
        #if not self.rounded:
        #    surface.fill(border,self.rect)
        #    surface.fill(color,self.rect.inflate(-4,-4))
        #else:
        if self.radius:
            rad = self.radius
        else:
            rad = 0
        self.round_rect(surface, self.rect , border, rad, 1, color)
        if self.text:
            text_rect = text.get_rect(center=self.rect.center)
            surface.blit(text,text_rect)
             
             
    def round_rect(self, surface, rect, color, rad=20, border=0, inside=(0,0,0,0)):
        rect = pygame.Rect(rect)
        zeroed_rect = rect.copy()
        zeroed_rect.topleft = 0,0
        image = pygame.Surface(rect.size).convert_alpha()
        image.fill((0,0,0,0))
        self._render_region(image, zeroed_rect, color, rad)
        if border:
            zeroed_rect.inflate_ip(-2*border, -2*border)
            self._render_region(image, zeroed_rect, inside, rad)
        surface.blit(image, rect)
 
 
    def _render_region(self, image, rect, color, rad):
        corners = rect.inflate(-2*rad, -2*rad)
        for attribute in ("topleft", "topright", "bottomleft", "bottomright"):
            pygame.draw.circle(image, color, getattr(corners,attribute), rad)
        image.fill(color, rect.inflate(-2*rad,0))
        image.fill(color, rect.inflate(0,-2*rad))
         
    def update(self):
        #for completeness
        pass
         
def button_was_pressed():
    print('button_was_pressed')

# Function to draw the "Forward" button
def draw_forward_button():
    forward_button_rect = pygame.Rect(100, HEIGHT - 125, 120, 30)
    pygame.draw.rect(screen, BLUE, forward_button_rect)
    font = pygame.font.SysFont(None, 24)
    text_surface = font.render('Forward', True, WHITE)
    screen.blit(text_surface, (125, HEIGHT - 120))
    return forward_button_rect

# Function to draw the "Backwards" button
def draw_backwards_button():
    backwards_button_rect = pygame.Rect(100, HEIGHT - 75, 120, 30)
    text_rect = pygame.draw.rect(screen, BLUE, backwards_button_rect)
    font = pygame.font.SysFont(None, 24)
    text_surface = font.render('Backwards', True, WHITE)
    text_rect = text_surface.get_rect(center=(100, HEIGHT - 75))
    screen.blit(text_surface, text_rect)
    return backwards_button_rect

# Function to draw the "Forward-Left" button
def draw_forward_left_button():
    forward_left_button_rect = pygame.Rect(WIDTH // 2 - 70, HEIGHT - 80, 100, 30)
    pygame.draw.rect(screen, BLUE, forward_left_button_rect)
    font = pygame.font.SysFont(None, 20)
    text_surface = font.render('Forward-Left', True, WHITE)
    screen.blit(text_surface, (WIDTH // 2 - 65, HEIGHT - 75))
    return forward_left_button_rect

# Function to draw the "Forward-Right" button
def draw_forward_right_button():
    forward_right_button_rect = pygame.Rect(WIDTH // 2 - 70, HEIGHT - 40, 100, 30)
    pygame.draw.rect(screen, BLUE, forward_right_button_rect)
    font = pygame.font.SysFont(None, 20)
    text_surface = font.render('Forward-Right', True, WHITE)
    screen.blit(text_surface, (WIDTH // 2 - 65, HEIGHT - 35))
    return forward_right_button_rect
    
# Function to draw the "Backwards-Left" button
def draw_backwards_left_button():
    backwards_left_button_rect = pygame.Rect(WIDTH // 2 + 40, HEIGHT - 80, 100, 30)
    pygame.draw.rect(screen, BLUE, backwards_left_button_rect)
    font = pygame.font.SysFont(None, 16)
    text_surface = font.render('Backwards-Left', True, WHITE)
    screen.blit(text_surface, (WIDTH // 2 + 45, HEIGHT - 75))
    return backwards_left_button_rect

# Function to draw the "Backwards-Right" button
def draw_backwards_right_button():
    backwards_right_button_rect = pygame.Rect(WIDTH // 2 + 40, HEIGHT - 40, 100, 30)
    pygame.draw.rect(screen, BLUE, backwards_right_button_rect)
    font = pygame.font.SysFont(None, 16)
    text_surface = font.render('Backwards-Right', True, WHITE)
    screen.blit(text_surface, (WIDTH // 2 + 45, HEIGHT - 35))
    return backwards_right_button_rect

# Function to draw the "Reset" button
def draw_reset_button():
    reset_button_rect = pygame.Rect(3 * WIDTH // 4 + 45, HEIGHT - 40, 100, 30)
    pygame.draw.rect(screen, BLUE, reset_button_rect)
    font = pygame.font.SysFont(None, 24)
    text_surface = font.render('Reset', True, WHITE)
    screen.blit(text_surface, (3 * WIDTH // 4 + 65, HEIGHT - 35))
    return reset_button_rect

def draw_calculate_button():
    calculate_button_rect = pygame.Rect(3 * WIDTH // 4 + 45, HEIGHT - 80, 100, 30)
    pygame.draw.rect(screen, BLUE, calculate_button_rect)
    font = pygame.font.SysFont(None, 24)
    text_surface = font.render('Calculate Path', True, WHITE)
    screen.blit(text_surface, (3 * WIDTH // 4 + 65, HEIGHT - 75))
    return calculate_button_rect

# Function to draw the obstacles
def draw_obstacles():
    for obs in obstacles:
        # Draw the obstacle itself as a black square
        obs_x, obs_y, direction = obs
        pygame.draw.rect(screen, BLACK, (obs_x * CELL_SIZE + 100, obs_y * CELL_SIZE + 50, CELL_SIZE, CELL_SIZE))
        
        # Highlight the grid line based on the direction
        if direction == 'n':  # North
            pygame.draw.line(screen, RED, (obs_x * CELL_SIZE + 100, obs_y * CELL_SIZE + 50), 
                             ((obs_x + 1) * CELL_SIZE + 100, obs_y * CELL_SIZE + 50), 4)
        elif direction == 's':  # South
            pygame.draw.line(screen, RED, (obs_x * CELL_SIZE + 100, (obs_y + 1) * CELL_SIZE + 50), 
                             ((obs_x + 1) * CELL_SIZE + 100, (obs_y + 1) * CELL_SIZE + 50), 4)
        elif direction == 'e':  # East
            pygame.draw.line(screen, RED, ((obs_x + 1) * CELL_SIZE + 100, obs_y * CELL_SIZE + 50), 
                             ((obs_x + 1) * CELL_SIZE + 100, (obs_y + 1) * CELL_SIZE + 50), 4)
        elif direction == 'w':  # West
            pygame.draw.line(screen, RED, (obs_x * CELL_SIZE + 100, obs_y * CELL_SIZE + 50), 
                             (obs_x * CELL_SIZE + 100, (obs_y + 1) * CELL_SIZE + 50), 4)
            
# Function to check movement is within 20x20 grid
def can_move(new_x, new_y):
    return 0 <= new_x <= GRID_SIZE - 3 and 0 <= new_y <= GRID_SIZE - 3

# Function to check if movement collides into obstacles
def is_collision(new_x, new_y):
    car_cells = [(new_x + dx, new_y + dy) for dx in range(robot_car_size) for dy in range(robot_car_size)]

    for obstacle in obstacles:
        x, y, face = obstacle

        for cell in car_cells:
            car_x, car_y = cell
        
            if car_x == x and car_y == y:
                return True
    
            for i in range(0, 1, 1):
                for j in range(0, 1, 1):
                    if car_x == x + i and car_y == y + j:
                        return True
    
    return False

# Function to move the robot car forward
def move_car_forward():
    global car_x, car_y
    new_x, new_y = car_x, car_y
    if current_direction == 0:  
        new_y -= 1
    elif current_direction == 1:
        new_x += 1
    elif current_direction == 2:
        new_y += 1
    elif current_direction == 3:
        new_x -= 1

    # Check if the movement forward is valid
    if can_move(new_x, new_y) and not is_collision(new_x, new_y):
        car_x, car_y = new_x, new_y
        
# Function to move the robot car backwards
def move_car_backwards():
    global car_x, car_y
    new_x, new_y = car_x, car_y
    if current_direction == 0:
        new_y += 1
    elif current_direction == 1:
        new_x -= 1
    elif current_direction == 2:
        new_y -= 1
    elif current_direction == 3:
        new_x += 1

    # Check if the movement backwards is valid
    if can_move(new_x, new_y) and not is_collision(new_x, new_y):
        car_x, car_y = new_x, new_y

# Function to move the robot car forward-left
def move_car_forward_left():
    global car_x, car_y, current_direction
    new_x, new_y = car_x, car_y

    # Move forward 3 steps
    if current_direction == 0:
        new_y -= TURN_RADIUS
    elif current_direction == 1:
        new_x += TURN_RADIUS
    elif current_direction == 2:
        new_y += TURN_RADIUS
    elif current_direction == 3:
        new_x -= TURN_RADIUS

    # Check if the movement forward is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Move left 3 steps
    if current_direction == 0: 
        new_x -= TURN_RADIUS
    elif current_direction == 1:
        new_y -= TURN_RADIUS
    elif current_direction == 2:  
        new_x += TURN_RADIUS
    elif current_direction == 3:  
        new_y += TURN_RADIUS

    # Check if the movement left is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Rotate counterclockwise if both checks passed
    car_x, car_y = new_x, new_y
    current_direction = (current_direction - 1) % 4  # Turn counterclockwise

# Function to move the robot car forward-right
def move_car_forward_right():
    global car_x, car_y, current_direction
    new_x, new_y = car_x, car_y
    
    # Move forward 3 steps 
    if current_direction == 0: 
            new_y -= TURN_RADIUS
    elif current_direction == 1: 
            new_x += TURN_RADIUS
    elif current_direction == 2:  
            new_y += TURN_RADIUS
    elif current_direction == 3:  
            new_x -= TURN_RADIUS

    # Check if the movement forward is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Move right 3 steps
    if current_direction == 0:  
        new_x += TURN_RADIUS
    elif current_direction == 1: 
        new_y += TURN_RADIUS
    elif current_direction == 2: 
        new_x -= TURN_RADIUS
    elif current_direction == 3: 
        new_y -= TURN_RADIUS

    # Check if the movement right is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Rotate clockise if both checks passed
    car_x, car_y = new_x, new_y
    current_direction = (current_direction + 1) % 4 

# Function to move the robot car backwards-left
def move_car_backwards_left():
    global car_x, car_y, current_direction
    new_x, new_y = car_x, car_y

    # Move backwards 3 steps
    if current_direction == 0:  
        new_y += TURN_RADIUS 
    elif current_direction == 1: 
        new_x -= TURN_RADIUS
    elif current_direction == 2:  
        new_y -= TURN_RADIUS
    elif current_direction == 3: 
        new_x += TURN_RADIUS

    # Check if the movement backwards is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Move left 3 steps
    if current_direction == 0:  
        new_x -= TURN_RADIUS
    elif current_direction == 1: 
        new_y -= TURN_RADIUS
    elif current_direction == 2:  
        new_x += TURN_RADIUS
    elif current_direction == 3:  
        new_y += TURN_RADIUS

    # Check if the movement left is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Rotate clockwise if both checks passed
    car_x, car_y = new_x, new_y
    current_direction = (current_direction + 1) % 4

# Function to move the robot car backwards-right
def move_car_backwards_right():
    global car_x, car_y, current_direction
    new_x, new_y = car_x, car_y

    # Move backwards 3 steps
    if current_direction == 0:  
        new_y += TURN_RADIUS
    elif current_direction == 1: 
        new_x -= TURN_RADIUS
    elif current_direction == 2:  
        new_y -= TURN_RADIUS
    elif current_direction == 3: 
        new_x += TURN_RADIUS

    # Check if the movement backwards is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Move right 2 steps
    if current_direction == 0: 
        new_x += TURN_RADIUS
    elif current_direction == 1: 
        new_y += TURN_RADIUS
    elif current_direction == 2:  
        new_x -= TURN_RADIUS
    elif current_direction == 3:  
        new_y -= TURN_RADIUS

    # Check if the movement right is valid
    if not can_move(new_x, new_y):
        print(f"Cannot move to {new_x}, {new_y}: Out of bounds or invalid.")
        return
    elif is_collision(new_x, new_y):
        print(f"Collision detected at {new_x}, {new_y}.")
        return

    # Rotate counterclockwise if both checks passed
    car_x, car_y = new_x, new_y
    current_direction = (current_direction - 1) % 4  
    
# Function to reset the robot car's position and direction
def reset_car():
    global car_x, car_y, current_direction
    car_x, car_y = 0, GRID_SIZE - 3 
    current_direction = 0  
            
# Main loop
running = True

def print_on_press():
    print('button pressed')

settings = {
    "clicked_font_color" : (0,0,0),
    "hover_font_color"   : (205,195, 100),
    'font'               : pygame.font.Font(None,16),
    'font_color'         : (255,255,255),
    'border_color'       : (0,0,0),
}
    
btn = Button(rect=(10,10,105,25), command=print_on_press, text='Press Me', **settings)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        btn.get_event(event)
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = event.pos
            grid_x = (mouse_pos[0] - 100) // CELL_SIZE  # Convert to grid coordinates
            grid_y = (mouse_pos[1] - 50) // CELL_SIZE

            if 0 <= grid_x < GRID_SIZE and 0 <= grid_y < GRID_SIZE:
                # Left-click (button 1) to add an obstacle
                if event.button == 1:
                    
                    deleted = False

                    for obs in obstacles:
                        if obs[0] == grid_x and obs[1] == grid_y:
                            deleted = True
                            obstacles.remove(obs)
                            break

                    if deleted == False:
                        obstacles.append((grid_x, grid_y, 'n'))
        
                # Right-click (button 3) to remove an obstacle
                elif event.button == 3:
                    # Find if there's an obstacle at the clicked position
                    directions = ['n', 'e', 's', 'w']
                    for obs in obstacles:
                        if obs[0] == grid_x and obs[1] == grid_y:
                            new = (obs[0], obs[1], directions[(directions.index(obs[2]) + 1) % 4])
                            obstacles.remove(obs)
                            obstacles.append(new)
                            break

            # Check if buttons were clicked
            forward_button_rect = draw_forward_button()
            backwards_button_rect = draw_backwards_button()
            forward_right_button_rect = draw_forward_right_button()
            forward_left_button_rect = draw_forward_left_button()
            backwards_left_button_rect = draw_backwards_left_button()
            backwards_right_button_rect = draw_backwards_right_button()  
            reset_button_rect = draw_reset_button()
            calculate_button_rect = draw_calculate_button()

            if forward_button_rect.collidepoint(mouse_pos):
                move_car_forward()
            if backwards_button_rect.collidepoint(mouse_pos):
                move_car_backwards()
            if forward_right_button_rect.collidepoint(mouse_pos):
                move_car_forward_right()
            if forward_left_button_rect.collidepoint(mouse_pos):
                move_car_forward_left()
            if backwards_left_button_rect.collidepoint(mouse_pos):
                move_car_backwards_left()
            if backwards_right_button_rect.collidepoint(mouse_pos):
                move_car_backwards_right()
            if reset_button_rect.collidepoint(mouse_pos):
                reset_car()
            if calculate_button_rect.collidepoint(mouse_pos):
                print("calculate!")

    # Fill the screen with white
    screen.fill(WHITE)

    # Draw the grid, car, and obstacles
    draw_grid()
    draw_car()
    draw_obstacles()

    # Draw the buttons
    draw_forward_button()
    draw_backwards_button()
    draw_forward_right_button()
    draw_forward_left_button()
    draw_backwards_left_button()
    draw_backwards_right_button()
    draw_calculate_button()
    draw_reset_button()
    btn.draw(screen)
    # Refresh the screen
    pygame.display.flip()
    


# Quit Pygame
pygame.quit()
sys.exit()


pygame 2.1.0 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

c:\Users\Abhinav T J\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
